In [ ]:
#simple direct problem solving 
import matplotlib.pyplot as plt

from clspde.solution import Solution
from clspde.basis import Basis
import itertools
import numpy as np


power = 3
params = {
        'n_dims': 2,
        'dim_sizes': np.array([2, 2]),
        'area_lims': np.array([[0,0.1], [0,1]]),
        'power': power,
        'basis': Basis(power),
        'n_funcs': 1,
    }

def prepare_model(border_weight, colloc_weight, connect_weights):
    eps = 0.5

    sol = Solution(**params)

    w = 1
    k1 = connect_weights[0]
    k2 = connect_weights[1]
    # border_weight = 1
    # no_need_weight = 0

    itercoef=0.

    colloc_left_operators = [lambda u_loc, u_bas, x, x_loc:  (-u_bas([1,0],0)-eps*u_bas([0,2],0)
                                                                                                +(u_bas([0,1],0)*u_loc([0,1],0))
                                                                                                ) * w**2*colloc_weight ,
                                                                                                ]


    def popravka(f,g):
        return f*g
    colloc_right_operators = [
                            lambda u_loc, u_nei, x, x_loc: (popravka(u_loc([0,0],0),  u_loc([0,0],0))/2) * w**2*colloc_weight,
                            ]
    colloc_ops = [colloc_left_operators, colloc_right_operators]


    def initial_state(x):
        t, x = x
        return x+1
        # sm_x = 3*x-1
        # if (x<2/3 and x>1/3):
        #     return 12 * (sm_x**2) *(1-sm_x)
        # else:
        #     return 0

    def terminal_state(x):
        t, x = x
        return -2*eps*np.log(x+1)
        # sm_x = 3*x-1
        # if (x<2/3 and x>1/3):
        #     return 12*sm_x*(1-sm_x)**2
        # else:
        #     return 0

    small = 1e-5

    def Bx(x):
        #border condition for x
        #check if t is greater than initial and lower than terminal
        return int((x[0]>sol.area_lims[0,0]+small) and (x[0]<sol.area_lims[0,1]-small))

    def BtI(x):
        #border condition Initial for t
        return int(x[0]==sol.area_lims[0,0])

    def BtT(x):
        #border condtition Terminal for t
        return int(x[0]>sol.area_lims[0,1]-small)

    border_left_operators = [
        lambda _, u_bas, x, x_loc: Bx(x)*(u_bas([0,0],1)) * border_weight,
        # lambda _, u_bas, x, x_loc: BtT(x)*(u_bas([0,0],0)-u_bas([0,0],1)) * border_weight, #* no_need_weight,
        lambda _, u_bas, x, x_loc: BtT(x)*(u_bas([0,0],1)) * border_weight, #* no_need_weight,
        
        # lambda _, u_bas, x, x_loc: BtI(x)*u_bas([0,0],0) * border_weight,

        lambda _, u_bas, x, x_loc: Bx(x)*(u_bas([0,0],0)) * border_weight,
        lambda _, u_bas, x, x_loc: BtI(x)*u_bas([0,0],0) * border_weight,
        ]

    border_right_operators = [
        lambda u, _, x, x_loc: Bx(x)* terminal_state(x) * border_weight, # border condition for psi
        # lambda u, _, x, x_loc: BtT(x) * (initial_state(x) - terminal_state(x)) * border_weight, #* no_need_weight,
        lambda u, _, x, x_loc: BtT(x) * (terminal_state(x)) * border_weight, #* no_need_weight,
        
        # lambda u, _, x, x_loc: BtI(x) * initial_state(x) * border_weight, # border and initial cond for s

        lambda u, _, x, x_loc: Bx(x)*initial_state(x) * border_weight,
        lambda u, _, x, x_loc: BtI(x)*initial_state(x) * border_weight,
    ]

    border_ops = [border_left_operators, border_right_operators]

    def f_collocation_points(N):
        points = np.zeros(N+1)
        h = 2/(N+1)
        points[0] = -1 + h/2
        for i in range(1, N+1):
            points[i] = points[i-1] + h
        return np.array(points).reshape(N+1,1)

    c_p_1d = f_collocation_points(power).reshape(power+1)

    colloc_points = np.array(list(itertools.product(c_p_1d, c_p_1d)))
    connect_points = np.array([
                                # [-1, 0.5], [1, 0.5],
                                # [0.5, -1], [0.5, 1],
                                # [-1, -0.5], [1, -0.5],
                                # [-0.5, -1], [-0.5, 1],
                                [-1, 0], [1, 0],
                                [0, -1], [0, 1],
                                ])
    border_points = np.array([
                                # [-1, 0.5], [1, 0.5],
                                # [0.5, -1], [0.5, 1],
                                # [-1, -0.5], [1, -0.5],
                                # [-0.5, -1], [-0.5, 1],
                                [-1, 0], [1, 0],
                                [0, -1], [0, 1],
                                # [1, 1], [1, -1],
                                # [-1, 1], [-1, -1],
                                ])

    
    points=[colloc_points, connect_points ,border_points]

    connect_left_operators = []
    connect_right_operators = []

    def dir(point: np.array) -> np.array:
        direction = (np.abs(point) == 1) * (np.sign(point))
        return np.array(direction, dtype=int)


    for func_num in range(sol.n_funcs):
        # connect_left_operators += [lambda _, u_bas, x, x_loc, func_num=func_num: u_bas(0*dir(x_loc), func_num) + np.sum(dir(x_loc)) * u_bas(dir(x_loc), func_num) * w,
                            # lambda _, u_bas, x, x_loc, func_num=func_num: u_bas(2*dir(x_loc), func_num)* w**2 + np.sum(dir(x_loc)) * u_bas(3*dir(x_loc), func_num)* w**3
                            # ]
        # connect_right_operators += [lambda _, u_nei, x, x_loc, func_num=func_num: u_nei(0*dir(x_loc), func_num) - np.sum(dir(x_loc))*u_nei(dir(x_loc), func_num)* w,
                                    # lambda _, u_nei, x, x_loc, func_num=func_num: u_nei(2*dir(x_loc), func_num)* w**2 - np.sum(dir(x_loc)) * u_nei(3*dir(x_loc), func_num)* w**3
                                    # ]
        connect_left_operators += [lambda _, u_bas, x, x_loc, func_num=func_num: k1*u_bas(0*dir(x_loc), func_num) + k2*np.sum(dir(x_loc))*u_bas([0,1],func_num)* np.all(dir(x_loc)==[0,1]),
                            # lambda _, u_bas, x, x_loc, func_num=func_num: u_bas(2*dir(x_loc), func_num)* w**2 + np.sum(dir(x_loc)) * u_bas(3*dir(x_loc), func_num)* w**3
                            ]
        connect_right_operators += [lambda _, u_nei, x, x_loc, func_num=func_num: u_nei(0*dir(x_loc), func_num) - np.sum(dir(x_loc))*u_nei([0,1],func_num)* np.all(dir(x_loc)==[0,-1]),
                                    # lambda _, u_nei, x, x_loc, func_num=func_num: u_nei(2*dir(x_loc), func_num)* w**2 - np.sum(dir(x_loc)) * u_nei(3*dir(x_loc), func_num)* w**3
                                    ]

    
    
    # for func_num in range(sol.n_funcs):
    #     connect_left_operators += [lambda _, u_bas, x, x_loc, func_num=func_num: (k1*u_bas(0*dir(x_loc),func_num) + k2*np.sum(dir(x_loc))*u_bas([0,1],func_num)*np.prod(dir(np.array([0.5,1]))==[0,1]))]
    #     connect_right_operators += [lambda _, u_bas, x, x_loc, func_num=func_num: (k1*u_bas(0*dir(x_loc),func_num) -  k2*np.sum(dir(x_loc))*u_bas([0,1],func_num)*np.prod(dir(np.array([0.5,1]))==[0,1]))]
    connect_ops = [connect_left_operators, connect_right_operators]

    iteration_dict = {'points':points,
            'colloc_ops':colloc_ops,
            'border_ops':border_ops,
        'connect_ops':connect_ops
    }

    sol.cells_coefs *= 0.0
    return sol, iteration_dict